# Imports

In [244]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load in dataset

In [245]:
df = pd.read_csv("../input/input.csv")

/var/folders/3m/1391qbxd3jg3m15k2d19w9qw0000gn/T/ipykernel_20721/3970109353.py:1: DtypeWarning: Columns (33,35,39,40,74,76,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../input/input.csv")


In [246]:
df.head()

,TEAM,RECORD_PCT,WINS,LOSSES,CONF_PCT,CONF_WINS,CONF_LOSSES,HOME_PCT,HOME_WINS,HOME_LOSSES,...,OPP_SOR,OPP_SOR_SCURVE,OPP_SOS,OPP_NCSOS,OPP_QUAL_WINS,OPP_QUAL_LOSSES,OPP_QUALITY_INDICATOR,OPP_FINAL_STREAK,OPP_SCORE,WINNER
0,Austin Peay Governors,0.686,24,11,0.686,16.0,4.0,0.686,12.0,1.0,...,7,7,25,13,11,6,7.117647,-1,74,1
1,Texas Longhorns,0.816,31,7,0.816,13.0,3.0,0.816,17.0,1.0,...,135,60,269,118,0,3,0.000000,-1,54,0
2,Austin Peay Governors,0.686,24,11,0.686,16.0,4.0,0.686,12.0,1.0,...,26,26,81,243,4,5,1.777778,-2,81,1
3,Vanderbilt Commodores,0.765,26,8,0.765,10.0,6.0,0.765,19.0,0.0,...,135,60,269,118,0,3,0.000000,-1,67,0
4,Austin Peay Governors,0.686,24,11,0.686,16.0,4.0,0.686,12.0,1.0,...,106,59,275,218,0,2,0.000000,-1,56,0


In [247]:
df = df.dropna()

# Predict Score

In [248]:
df.NCSOS = df.NCSOS.apply(pd.to_numeric, errors="coerce")
df.OPP_NCSOS = df.OPP_NCSOS.apply(pd.to_numeric, errors="coerce")
df.FINAL_STREAK = df.FINAL_STREAK.apply(pd.to_numeric, errors="coerce")
df.OPP_FINAL_STREAK = df.OPP_FINAL_STREAK.apply(pd.to_numeric, errors="coerce")
df.SCORE = df.SCORE.apply(pd.to_numeric, errors="coerce")
df.OPP_SCORE = df.OPP_SCORE.apply(pd.to_numeric, errors="coerce")

In [249]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df.CONF)
df.CONF = le.transform(df.CONF)
df.OPP_CONF = le.transform(df.OPP_CONF)

In [250]:
df = df.drop(columns=[
#     "OPP_SCORE", 
#                       "SCORE",
                      "WINNER",
                      "TEAM",
                      "OPP_TEAM",
                      "SOR_SCURVE",
                      "OPP_SOR_SCURVE",
                      "FINAL_STREAK",
                      "OPP_FINAL_STREAK"])
# Drop columns that don't contrubute much to the predictions
df = df.drop(columns=[
    "VS_AP_WINS",
    "OPP_VS_AP_WINS",
    "VS_AP_LOSSES",
    "OPP_VS_AP_LOSSES",
    "AWAY_WINS",
    "OPP_AWAY_WINS",
    "AWAY_LOSSES",
    "OPP_AWAY_LOSSES",
    "HOME_WINS",
    "OPP_HOME_WINS",
    "HOME_LOSSES",
    "OPP_HOME_LOSSES",
    "CONF",
    "OPP_CONF",
    "QUAL_WINS",
    "QUAL_LOSSES",
    "OPP_QUAL_WINS",
    "OPP_QUAL_LOSSES",
    "CONF_WINS",
    "CONF_LOSSES",
    "OPP_CONF_WINS",
    "OPP_CONF_LOSSES",
])
df = df.dropna()
# y = df.WINNER
y = df[["SCORE", "OPP_SCORE"]]
# y = pd.Dataframe({"score": df.SCORE, "opp_score": df.OPP_SCORE})
# x = df.drop(columns=["WINNER"])
x = df.drop(columns=["SCORE", "OPP_SCORE"])


In [251]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=1234)

In [256]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
# rf = RandomForestClassifier(
#     n_estimators=1500,
#     min_samples_leaf=4,
#     max_features='auto',
#     max_depth=50,
#     bootstrap=True)
rf = RandomForestRegressor(
    n_estimators=1500,
    min_samples_leaf=4,
    max_features='auto',
    max_depth=50,
    bootstrap=True
)
rf.fit(train_x, train_y)

# Log loss was the metric used in the Kaggle competition
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import mean_absolute_error

/Users/regbert/rf/application/venv38/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


In [272]:
### Random Forest
rf_pred = rf.predict(test_x)
# rf_acc = accuracy_score(test_y, rf_pred)
# rf_mae = mean_absolute_error(test_y, rf_pred)
# print(f"{'Random Forest MAE:':<20} ==> {rf_mae}")
# rf_predprob = rf.predict_proba(test_x)
# print(f"{'Random Forest':<20} ==> {round(rf_acc * 100, 2)}")

In [270]:
### Logistic Regression
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
log.fit(train_x, train_y)

ValueError: y should be a 1d array, got an array of shape (97489, 2) instead.

In [271]:
log_pred = log.predict(test_x)
log_acc = accuracy_score(test_y, log_pred)
log_mae = mean_absolute_error(test_y, log_pred)
print(f"{'Logistic Regression MAE:':<20} ==> {log_mae}")
log_predprob = log.predict_proba(test_x)
print(f"{'Logistic Regression':<20} ==> {round(log_acc * 100, 2)}")

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

In [274]:
### ANN
from sklearn.neural_network import MLPClassifier, MLPRegressor
# ann = MLPClassifier(hidden_layer_sizes = (50,100,25,20))
ann = MLPRegressor(hidden_layer_sizes = (50,100,25,20))
ann.fit(train_x, train_y)

MLPRegressor(hidden_layer_sizes=(50, 100, 25, 20))

In [281]:
### ANN
ann_pred = ann.predict(test_x)
# ann_acc = accuracy_score(test_y, ann_pred)
ann_mae = mean_absolute_error(test_y, ann_pred)
print(f"{'Neural Net MAE:':<20} ==> {ann_mae}")
# ann_predprob = ann.predict_proba(test_x)
# print(f"{'Neural Net':<20} ==> {round(ann_acc * 100, 2)}")

Neural Net MAE:      ==> 7.593390237583449


In [258]:
# Linear Regression
from sklearn.linear_model import LinearRegression
lin = LinearRegression()
lin.fit(train_x, train_y)

LinearRegression()

In [259]:
from sklearn.metrics import mean_absolute_error
lin_pred = lin.predict(test_x)
lin_acc = lin.score(test_x, test_y)
lin_mae = mean_absolute_error(test_y, lin_pred)
print(f"{'Linear Regression MAE:':<20} ==> {lin_mae}")
print(f"{'Linear Regression (r-squared)':<20} ==> {lin_acc}")

Linear Regression MAE: ==> 7.5498096763301685
Linear Regression (r-squared) ==> 0.40861809163776525


In [260]:
lin_pred[:10]

array([[71.97505313, 71.17264902],
       [69.64731666, 85.28729777],
       [74.26335908, 57.37965378],
       [71.51925209, 66.95508359],
       [58.87390113, 70.40957209],
       [74.41351676, 54.96028753],
       [76.27560351, 75.26841313],
       [73.79836661, 72.41906857],
       [69.46529143, 57.50105662],
       [74.11323436, 69.62829697]])

In [262]:
test_x[:10]

,RECORD_PCT,WINS,LOSSES,CONF_PCT,HOME_PCT,AWAY_PCT,PPG,ORBPG,DRBPG,APG,...,OPP_FGAPG,OPP_FTAPG,OPP_3PAPG,OPP_OBPI,OPP_DBPI,OPP_BPIRK,OPP_SOR,OPP_SOS,OPP_NCSOS,OPP_QUALITY_INDICATOR
99137,0.636,21,12,0.636,0.636,0.636,73.3,13.062500,26.500000,12.3,...,54.781250,23.343750,18.437500,2.8,4.4,61,89,30,57.0,1.785714
18636,0.400,12,18,0.400,0.400,0.400,72.8,12.066667,23.366667,14.1,...,59.781250,22.218750,14.968750,3.7,0.2,101,97,82,274.0,0.571429
87778,0.375,12,20,0.375,0.375,0.375,66.5,9.687500,25.031250,12.2,...,56.064516,21.387097,18.967742,-4.6,-1.7,271,267,205,165.0,0.000000
135602,0.667,20,10,0.667,0.667,0.667,72.2,10.700000,25.833333,13.4,...,57.205882,16.264706,23.911765,-3.2,-0.8,237,257,223,109.0,0.000000
20933,0.207,6,23,0.207,0.207,0.207,60.0,11.413793,23.448276,12.6,...,57.774194,22.838710,15.870968,1.4,0.3,139,136,182,22.0,0.000000
42950,0.484,15,16,0.484,0.484,0.484,64.6,10.838710,23.064516,12.3,...,54.928571,19.821429,16.750000,-5.6,-7.6,333,311,338,99.0,0.000000
128010,0.632,24,14,0.632,0.632,0.632,80.0,12.441176,26.764706,15.2,...,57.314286,25.285714,16.857143,-3.9,0.8,220,185,171,202.0,0.000000
128901,0.545,18,15,0.545,0.545,0.545,71.4,10.454545,27.303030,13.6,...,63.181818,20.121212,25.303030,-3.3,-1.4,258,236,234,176.0,0.000000
148889,0.688,22,10,0.688,0.688,0.688,66.1,8.400000,24.100000,14.0,...,58.966667,15.366667,21.133333,-2.4,-0.4,225,210,123,278.0,0.000000
93241,0.676,23,11,0.676,0.676,0.676,71.8,12.151515,29.121212,14.2,...,56.033333,24.966667,23.666667,2.9,-4.1,188,151,256,237.0,0.000000


In [264]:
test_y[:10].to_numpy()

array([[62., 59.],
       [66., 75.],
       [61., 43.],
       [81., 71.],
       [63., 79.],
       [72., 63.],
       [94., 85.],
       [48., 58.],
       [84., 57.],
       [76., 63.]])

In [ ]:
# Feature importances
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)

In [ ]:
feature_names = list(df.columns[:-1])
forest_importances = pd.Series(importances, index=feature_names).sort_values()
fig, ax = plt.subplots(figsize=(10, 6))
forest_importances.plot.bar(ax=ax)
ax.set_title("Variable Importances")
ax.set_ylabel("Impurity Decrease")
fig.tight_layout()

In [268]:
def analyze_predicted_scores(predictions, actual_scores):
    total_scores = len(predictions)
    if total_scores != len(actual_scores):
        print("Array lengths don't line up!")
        return -1
    correct = 0
    for i in range(total_scores):
        pred = predictions[i]
        score = actual_scores[i]
        if pred[0] > pred[1] and score[0] > score[1] or \
            pred[0] < pred[1] and score[0] < score[1]:
            correct += 1
#         else:
#             print(f"Incorrect: Prediction({pred}) --> Score({score})")
            
    return correct/total_scores

In [283]:
analyze_predicted_scores(lin_pred, test_y.to_numpy())

0.747343371763837

In [284]:
analyze_predicted_scores(rf_pred, test_y.to_numpy())

0.7270340130472244

In [285]:
analyze_predicted_scores(ann_pred, test_y.to_numpy())

0.7552619702129405